In [1]:
from IPython.display import clear_output
from functions import *

import os
import json

In [2]:
competitions = ['CdB', 'Serie_A', 'Serie_B', 'Serie_C', 'Serie_D']

with open('number_of_games.json') as f:
    n_games = json.load(f)

In [3]:
%%time
errors = {}
cont_sucess = 0
cont_fail = 0
for competition in competitions:
    for year in range(2013, 2023):
        clear_output()
        print(f'Iniciando o ano de {year} para {competition.replace("_", " ")}')
        games = {}
        for game in range(1, n_games[competition][str(year)] + 1):
            f_club, f_result, f_players, f_goals, f_changes = False, False, False, False, False
            try:
                file = f'{competition}/{str(year)}/CSVs/{str(game).zfill(3)}.csv'
                f = open(file)
                data = f.readlines()
                f.close()
                text = ''
                for row in data:
                    text += row
                    
                clubs = catch_teams(text)
                assert len(clubs) == 1
                f_club = True
                
                result = final_result(text)
                assert len(result) == 1
                result = result[0]
                f_result = True
                
                players = catch_players(text)
                assert len(players) >= 36
                f_players = True
                
                goals = catch_goals(text)
                assert len(goals) == int(result[0]) + int(result[-1])
                f_goals = True
                
                changes = find_changes(text)
                assert len(changes) <= 10
                f_changes = True
                
                games[str(game).zfill(3)] = {'Mandante'      : clubs[0][0],
                                             'Visitante'     : clubs[0][1],
                                             'Resultado'     : result,
                                             'Jogadores'     : players,
                                             'Gols'          : goals,
                                             'Substituições' : changes}
                
                cont_sucess += 1
            except FileNotFoundError:
                erro = 'scrape'
                if competition in errors:
                    if f'{year}' in errors[competition]:
                        errors[competition][f'{year}'][str(game).zfill(3)] = erro
                    else:
                        errors[competition][f'{year}'] = {}
                        errors[competition][f'{year}'][str(game).zfill(3)] = erro
                else:
                    errors[competition] = {}
                    errors[competition][f'{year}'] = {}
                    errors[competition][f'{year}'][str(game).zfill(3)] = erro
            
            except AssertionError:
                cont_fail += 1
                if not f_club:
                    erro = 'clube'
                elif not f_result:
                    erro = 'resultado'
                elif not f_players:
                    erro = 'jogadores'
                elif not f_goals:
                    erro = 'gols'
                elif not f_changes:
                    erro = 'substituições'
                else:
                    erro = 'ver'
                if competition in errors:
                    if f'{year}' in errors[competition]:
                        errors[competition][f'{year}'][str(game).zfill(3)] = erro
                    else:
                        errors[competition][f'{year}'] = {}
                        errors[competition][f'{year}'][str(game).zfill(3)] = erro
                else:
                    errors[competition] = {}
                    errors[competition][f'{year}'] = {}
                    errors[competition][f'{year}'][str(game).zfill(3)] = erro

        with open(f'{competition}/{str(year)}/games.json', 'w') as f:
            json.dump(games, f)

clear_output()
print(f'Extração finalizada.\n{cont_sucess} jogos foram extraídos com sucesso.\n{cont_fail} jogos falharam.')

Extração finalizada.
13459 jogos foram extraídos com sucesso.
320 jogos falharam.
CPU times: user 37 s, sys: 365 ms, total: 37.4 s
Wall time: 37.3 s
